Part One

In [1]:
import numpy as np
import galois
from itertools import product

In [2]:
with open("input10.txt", "r") as f:
    manual = f.readlines()

targets, all_buttons = [line.split()[0].strip('[]') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [tuple(0 if ch=="." else 1 for ch in t) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

sum_min_presses = 0
for target, buttons in zip(targets,all_buttons):
    target, buttons = np.array(target).reshape(-1,1), np.array(buttons)
    matrix = galois.GF(2)(np.hstack((buttons.T, target)))
    RREF = matrix.row_reduce()

    #from RREF, gets a dictionary with the possible values of the independent variables and
    #and another with the dependence relations of the dependent ones
    independent_vars, dependent_vars = {}, {}
    for row in RREF[::-1]:
        if (row == 0).all(): continue
        whereone = np.argwhere(row[:-1])
        if whereone.size==1: independent_vars[int(whereone[0,0])]={int(row[-1])}; continue
        for j in whereone[1:].reshape(-1,): independent_vars[int(j)] = {0,1}
        dependent_vars[int(whereone[0,0])] = (tuple(int(i) for i in whereone[1:].reshape(-1,)),int(row[-1]))

    #first an iterable only with tuples filled with the independent vars (elsewhere placeholders)
    pre_possibles = product(*[{j for j in independent_vars[i]} if i in independent_vars else {-1} for i in range(RREF.shape[1]-1)])
    #now populate with all possible tuples
    possibles = set()
    for poss in pre_possibles:
        poss_lst = list(poss)
        for i in dependent_vars:
            poss_lst[i] = (sum(poss[j] for j in dependent_vars[i][0])+dependent_vars[i][1]) %2
        possibles.add(tuple(poss_lst))
    del pre_possibles
    sum_min_presses+= min(sum(poss) for poss in possibles)
sum_min_presses

447

Around 4.5s

Part Two

Now without galois GF(2) fields. Also with allowed ranges for the independent variables

In [1]:
import numpy as np
from itertools import product
import sympy as sp

In [ ]:
with open("input10.txt", "r") as f:
    manual = f.readlines()
# manual = manual[-1:] #for testing
targets, all_buttons = [line.split('{')[1].strip('}\n') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [tuple(int(i) for i in t.split(',')) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

sum_min_presses = 0
for inst,(target, buttons) in enumerate(zip(targets,all_buttons)):
    print(f"instance {inst}", end="\r")

    target, buttons = sp.Matrix(target), sp.Matrix(buttons)
    matrix = sp.Matrix.hstack(buttons.T, target)
    RREF = matrix.rref()[0]
    
    #from RREF, gets a dictionary with the possible values of the independent variables and
    #and another with the dependence relations of the dependent ones
    independent_vars, dependent_vars = {}, {}
    for i in range(RREF.shape[0]):
        row = RREF[i,:]
        wherenonzero = np.argwhere(row[:-1])
        if wherenonzero.size == 0: continue
        if wherenonzero.size==1: independent_vars[int(wherenonzero[0,0])]=[int(row[-1])]; continue
        dependent_vars[int(wherenonzero[0,0])] = (row[-1],tuple((-row[i],int(i)) for i in wherenonzero[1:].reshape(-1,)))
        for j in wherenonzero[1:].reshape(-1,):
            if int(j) not in independent_vars: independent_vars[int(j)] = [0,np.inf]
        if wherenonzero.size==2:
            if row[wherenonzero[1,0]]>0:
                upper_bound = row[-1]//row[wherenonzero[1,0]]
                independent_vars[wherenonzero[1,0]][1] = min(upper_bound, independent_vars[wherenonzero[1,0]][1])
            elif row[-1]<0:
                lower_bound = -(row[-1]//row[wherenonzero[1,0]])
                independent_vars[wherenonzero[1,0]][0] = max(lower_bound, independent_vars[wherenonzero[1,0]][0])

    #tightening bounds on independent variables, essential to reduce from 2h to 25s
    #bounding the inf upper bounds
    unbounded = {i for i in independent_vars if independent_vars[i][-1]==np.inf}
    for i in unbounded:
        independent_vars[i][-1] = min({matrix[ii,-1] for ii in range(matrix.shape[0]) if matrix[ii,i]})
    #tightening from dependent-variable equations (running once is good enough, no need to run while changes)
    for lone_term,dependencies in dependent_vars.values():
        for coeff, var in dependencies:
            #same expression for upper and lower bounds, even though different signs and rounding
            bound = (lone_term + sum(c*independent_vars[v][(-np.sign(c)-1)//2] for c,v in dependencies if v!=var) )//-coeff
            if coeff<0: independent_vars[var][1] = min(bound,independent_vars[var][1])
            else: independent_vars[var][0] = max(bound,independent_vars[var][0]) #coeff>0
    #now range format
    independent_vars = {k:range(v[0],v[-1]+1) for k,v in independent_vars.items()}

    #first an iterable only with tuples filled with the independent vars (elsewhere placeholders)
    pre_possibles = product(*[{j for j in independent_vars[i]} if i in independent_vars else {np.nan} for i in range(RREF.shape[1]-1)])
    #now populate with all possible tuples
    possibles = set()
    for poss in pre_possibles:
        poss_lst = list(poss)
        for i in dependent_vars:
            poss_lst[i] = dependent_vars[i][0] + sum(coeff*poss[idx] for coeff,idx in dependent_vars[i][1])
        if all(v//1==v and v >= 0 for v in poss_lst):
            possibles.add(tuple(poss_lst))
    del pre_possibles
    # print(min({sum(poss) for poss in possibles}))
    sum_min_presses += min({sum(poss) for poss in possibles})

print("Final sum is", sum_min_presses)

Final sum is 18960


Decent time, ~25s

Feels like cheating: LP solvers + branch-and-bound

In [3]:
import numpy as np
import cvxpy as cp

In [4]:
with open("input10.txt", "r") as f:
    manual = f.readlines()
# manual = manual[:3] #for testing
targets, all_buttons = [line.split('{')[1].strip('}\n') for line in manual], [line.split("]")[1].split("{")[0].strip().split() for line in manual]
targets, all_buttons = [tuple(int(i) for i in t.split(',')) for t in targets], [[tuple(map(int,v.strip("()").split(","))) for v in lst] for lst in all_buttons]
all_buttons = [[tuple(1 if i in v else 0 for i in range(len(k))) for v in lst] for k,lst in zip(targets,all_buttons)]

EPS = .00001

sum_min_presses = 0
for inst,(target, buttons) in enumerate(zip(targets,all_buttons)):
    print(f"target {inst}:", target, end="                \r")

    target, buttons = np.array(target), np.array(buttons).T
    x = cp.Variable(buttons.shape[1])
    # buttons_cp = cp.Parameter(buttons.shape); buttons_cp.value = buttons
    objective = cp.Minimize(cp.sum(x))
    constraints = [buttons@x == target, x>=0]
    problem = cp.Problem(objective, constraints)
    problem.solve("SCIPY")
    if problem.status == 'optimal' and np.allclose(x.value, np.round(x.value)): sum_min_presses+=int(np.round(problem.value)); continue
    #else:branch
    non_int_idx = np.argwhere(abs(x.value-np.round(x.value))>EPS)[0]
    int_down, int_up = int(x.value[non_int_idx][0]), int(x.value[non_int_idx][0])+1
    branches = [(problem.value, [x[non_int_idx] <= int_down], x.value), (problem.value, [x[non_int_idx] >= int_up], x.value)]
    upper_bound = np.inf
    while branches:
        _, branch_constraints, __ = min(branches, key=lambda br:br[0])
        branches.remove((_,branch_constraints,__))
        problem = cp.Problem(objective, constraints+branch_constraints)
        problem.solve("SCIPY")
        if problem.status == 'optimal' and np.allclose(x.value, np.round(x.value)):
            upper_bound = min(problem.value, upper_bound)
            branches = [br for br in branches if br[0]<upper_bound]
        else:
            if problem.status == 'infeasible': continue
            non_int_idx = np.argwhere(abs(x.value-np.round(x.value))>EPS)[0]
            int_down, int_up = int(x.value[non_int_idx][0]), int(x.value[non_int_idx][0])+1
            branches.append((problem.value, branch_constraints+[x[non_int_idx] <= int_down], x.value))
            branches.append((problem.value, branch_constraints+[x[non_int_idx] >= int_up], x.value))
    sum_min_presses+=int(np.round(upper_bound))

sum_min_presses

18960

Around 7s. But it feels like cheating...

###### little test shows how galois reduce_row is really different from straight rref

In [ ]:
import sympy as sp
for target, buttons in zip(targets[1:2],all_buttons):
    target, buttons = np.array(target).reshape(-1,1), np.array(buttons)
    matrix = galois.GF(2)(np.hstack((buttons.T, target)))
    RREF = matrix.row_reduce()
    print(RREF)
    print(sp.Matrix.hstack(sp.Matrix(buttons.T), sp.Matrix(target)).rref())

[[1 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 1 0]
 [0 0 1 0 0 0 0 0 0 0 1 0]
 [0 0 0 1 0 0 0 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 1 0]
 [0 0 0 0 0 1 0 0 0 0 1 1]
 [0 0 0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 1 1]
 [0 0 0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 1 1 0]]
(Matrix([
[1, 0, 0, 0, 0, 0, 0, 0, 0, 0,  0,    0],
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0,  1,    0],
[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, -1, -2/3],
[0, 0, 0, 1, 0, 0, 0, 0, 0, 0,  0, -1/3],
[0, 0, 0, 0, 1, 0, 0, 0, 0, 0,  1,  2/3],
[0, 0, 0, 0, 0, 1, 0, 0, 0, 0,  1,  7/3],
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0,  0, -2/3],
[0, 0, 0, 0, 0, 0, 0, 1, 0, 0,  1,  1/3],
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, -1,  1/3],
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1,   -2]]), (0, 1, 2, 3, 4, 5, 6, 7, 8, 9))
